In [1]:
import numpy as np
from scipy.stats import uniform, norm, f, t
import pandas as pd
import statsmodels.api as sm
np.random.seed(314)


In [2]:
n = 30
param = 5
X = uniform.rvs(loc = np.array([[-1,-3,1,1,-4][:param]]), scale =  np.array([[2,4,9,9,3][:param]]), size = (n,param))
b = np.array([[4, -3, 0.5, 3, 0][:param]]).T
cons = 3
model_error = norm.rvs(loc=0, scale = 1, size= (n,1))
y = cons + np.matmul(X,b)+model_error

In [3]:
X = pd.DataFrame(X)
X = sm.add_constant(X)

df residual = no. observations - no. parameters (intercept is counted)

df model = no. regressors (intercept is not counted)

In [4]:
dof_res = len(X)-len(X.columns)
dof_model = len([col for col in X.columns if col!="const"])
print("dof_res: {}, dof_model: {}".format(dof_res, dof_model))

dof_res: 24, dof_model: 5


In [5]:
est = sm.OLS(y,X)
est_fit = est.fit()
print(est_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     603.3
Date:                Fri, 09 Jun 2023   Prob (F-statistic):           2.10e-24
Time:                        18:51:05   Log-Likelihood:                -35.961
No. Observations:                  30   AIC:                             83.92
Df Residuals:                      24   BIC:                             92.33
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.4325      0.755      4.545      0.0

In [6]:
lim_inf = t.ppf(0.025,est_fit.df_resid)*est_fit.bse+est_fit.params
lim_sup = t.ppf(0.975,est_fit.df_resid)*est_fit.bse+est_fit.params
tvalue= est_fit.params/est_fit.bse
pt = tvalue.apply(lambda x: 2 - 2*t.cdf(abs(x),est_fit.df_resid))
pd.DataFrame({"t":tvalue, "p>|t|":pt,"ci_inf":lim_inf,"ci_sum":lim_sup})

,t,p>|t|,ci_inf,ci_sum
const,4.544802,1.321524e-04,1.873726,4.991285
0,14.540670,2.131628e-13,3.453906,4.596592
1,-18.210840,1.554312e-15,-3.030118,-2.413207
2,4.646509,1.020226e-04,0.211314,0.549059
3,45.033457,0.000000e+00,2.863857,3.138969
4,-1.238184,2.276232e-01,-0.673719,0.168469


In [7]:
{"model_sse":((est_fit.fittedvalues-y.mean())**2).sum(),
"residuals_sse":((est_fit.fittedvalues-y.squeeze())**2).sum(),
"total_sse":((y.mean()-y)**2).sum()}

{'model_sse': 2427.2334575486807,
 'residuals_sse': 19.311895268611927,
 'total_sse': 2446.54535281729}

In [8]:
#MSE
dfe = pd.DataFrame({"model":{"SME":est_fit.mse_model,"df":est_fit.df_model},
              "residuals":{"SME":est_fit.mse_resid,"df":est_fit.df_resid},
              "total":{"SME":est_fit.mse_total,"df":est_fit.df_model+est_fit.df_resid}
            }).T
dfe["SSE"]= dfe["SME"]*dfe["df"]

dfe

,SME,df,SSE
model,485.446692,5.0,2427.233458
residuals,0.804662,24.0,19.311895
total,84.363633,29.0,2446.545353


In [9]:
#R_squared
1-dfe.loc["residuals","SSE"]/dfe.loc["total","SSE"]

0.9921064634071167

In [10]:
#F-statistic
f_value = dfe.loc["model","SME"]/dfe.loc["residuals","SME"]

pf = 1 - f.cdf(f_value,dfe.loc["model","df"],dfe.loc["residuals","df"])

print("f={}, model_df = {}, residual_df = {} -> P>f = {}".format(f_value, dfe.loc["model","df"],dfe.loc["residuals","df"],pf))

f=603.2924492486163, model_df = 5.0, residual_df = 24.0 -> P>f = 1.1102230246251565e-16


In [14]:
print("The real value of the first observation is {}, we have predicted {}".format(y[0,0],est_fit.predict(X.iloc[0]).iloc[0]))

The real value of the first observation is 35.36942031398361, we have predicted 34.30606626720518


In [15]:
#Mean 95% CI estimation
est_fit.get_prediction(X.iloc[0]).conf_int(obs=False)

array([[33.40933472, 35.20279781]])

In [16]:
#Value 95% CI estimation
est_fit.get_prediction(X.iloc[0]).conf_int(obs=True)

array([[32.24894955, 36.36318298]])

In [17]:
#std err - lr 1 independent variable. some correlation terms lack for other cases
np.sqrt(est_fit.mse_resid/((X - X.mean(axis=0))**2).sum(axis=0))[0]

0.2607257169393112